<a href="https://colab.research.google.com/github/ZeynepRuveyda/ADA/blob/main/Blemish_removal_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DATA_PATH = '/content/drive/My Drive/OpenCV_Documentation/week6/data6/'

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import matplotlib

In [ ]:
def calculateVariance(patch):
    a = cv2.Scharr(patch, -1, 1, 0)
    b = cv2.Scharr(patch, -1, 0, 1)

    return np.abs(a) + np.abs(b)

In [ ]:
def displayZoom(fragment, wndName, wndTitle=None):
    size = (400, 400)
    resizedFragment = cv2.resize(fragment, size)
    if wndTitle != None:
        cv2.setWindowTitle(wndName, wndTitle)
    cv2.imshow(wndName, resizedFragment)

In [ ]:
def getPatch(ab, img):
    patch = img[ab[1] - 15:ab[1] + 15, ab[0] - 15:ab[0] + 15]
    return patch

In [ ]:

def pickBestAround(ab, values, image):
    bestV = 0
    best_ab = None
    for move in np.array([(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]) * 30:
        ab_m = ab + move
        if ab_m[ab_m<0].sum() < 0:
            continue
        patch = getPatch(ab_m, values)
        variance = 1/calculateVariance(patch).sum()
        if variance > bestV:
            bestV = variance
            best_ab = ab_m

        return getPatch(best_ab, values), getPatch(best_ab, image)

In [ ]:
def onMouse(action, a, b, flags, img):
    if action == cv2.EVENT_LBUTTONDOWN or action == cv2.EVENT_RBUTTONDOWN:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:, :, 0]

        # pick the best patch and display it
        (grayPatch, colorPatch) = pickBestAround((a, b), gray, img)
        
        # seamless clone the patch onto the image
        src_mask = np.ones_like(grayPatch) * 255
        cv2.seamlessClone(
            colorPatch, img, src_mask, (a, b), cv2.NORMAL_CLONE, blend=img)
        
        if action == cv2.EVENT_RBUTTONDOWN:
          gray = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)[:, :, 0]
          # display the selected area
          patch, cpatch = getPatch((a,b), gray), getPatch((a,b), img)

          # pick the best patch and display it
          (grayPatch, colorPatch) = pickBestAround((a, b), gray, img)

          variance = calculateVariance(patch)
          displayZoom(cpatch, "Variance",
                      "Patch {0}".format(np.sum(variance)))
          variance = calculateVariance(grayPatch)
          displayZoom(colorPatch, "Best Patch",
                      "Best Patch V={0}".format(np.sum(variance)))

In [ ]:
imagePath = DATA_PATH + '/blemish.png'
img = cv2.imread(imagePath)

# Make a dummy image, will be useful to clear the drawing
cv2.namedWindow("Window")
# highgui function called when mouse events occur
cv2.setMouseCallback("Window", onMouse, img)



In [ ]:
k = 0
while k != 27:
    cv2.imshow("Window", img)
    k = cv2.waitKey(20)

cv2.destroyAllWindows()